In [22]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('RFM_ht_data.csv', parse_dates=['InvoiceDate'])

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [26]:
df.CustomerCode = df.CustomerCode.apply(str)

In [27]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [28]:
df.InvoiceNo = df.InvoiceNo.apply(str)

In [29]:
df.shape

(332730, 4)

In [30]:
df.groupby('CustomerCode')\
  .InvoiceNo.nunique()\
  .sort_values(ascending=False)

CustomerCode
19057820    204
13215452    113
13032521    106
19080880     99
99003061     90
           ... 
13272861      1
13272871      1
13272875      1
13272878      1
14000778      1
Name: InvoiceNo, Length: 123733, dtype: int64

In [31]:
df.CustomerCode.value_counts()

19057820    204
13215452    113
13032521    106
19080880     99
99003061     90
           ... 
19061688      1
77038884      1
19066032      1
70029827      1
13046679      1
Name: CustomerCode, Length: 123733, dtype: int64

In [33]:
last_date = df.InvoiceDate.max()

In [35]:
rfmTable = df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days,
                                           'InvoiceNo': lambda x: len(x),
                                           'Amount': lambda x: x.sum()})


In [37]:
rfmTable.head()

,InvoiceDate,InvoiceNo,Amount
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [36]:
rfmTable.InvoiceDate = rfmTable.InvoiceDate.apply(int)

In [39]:
rfmTable.rename(columns={'InvoiceDate': 'recency',
                         'InvoiceNo': 'frequency',
                         'Amount': 'monetary'}, inplace=True)

In [40]:
rfmSegmentation = rfmTable.copy()

In [45]:
quantiles = rfmTable.quantile(q=[0.25, 0.5, 0.75])

In [46]:
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [41]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [44]:
rfmTable.frequency.describe()

count    123733.000000
mean          2.689097
std           2.894494
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max         204.000000
Name: frequency, dtype: float64

In [48]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary'].apply(FMClass, args=('monetary',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) + rfmSegmentation.F_Quartile.map(str) + rfmSegmentation.M_Quartile.map(str)

In [49]:
rfmSegmentation

,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...
99099927,10,1,961.10,3,4,3,343
99099936,0,1,1521.78,1,4,3,143
99099959,8,2,1444.56,2,3,3,233


In [56]:
rfmSegmentation.query('RFMClass == "311"')\
               .groupby(['R_Quartile', 'F_Quartile', 'M_Quartile'])\
               .agg({'recency': 'count'})

,,,recency
R_Quartile,F_Quartile,M_Quartile,
3,1,1,1609


In [60]:
rfmSegmentation.groupby(['R_Quartile', 'F_Quartile', 'M_Quartile'])\
               .agg({'recency': 'count'})\
               .min()

recency    2
dtype: int64

In [62]:
df.InvoiceDate.max()

Timestamp('2020-09-30 00:00:00')